In [1]:
%load_ext Cython

In [1]:
%%cython --annotate

from libc.math cimport sqrt, exp, pow
import time

# def memoize(func):
#     cache = dict()

#     def memoized_func(*args):
#         if args in cache:
#             return cache[args]
#         result = func(*args)
#         cache[args] = result
#         return result

#     return memoized_func

t1 = time.time()
cdef float max_(float a, float b) except? -2:
    if a > b:
        return a
    else:
        return b

cdef float volatility = 0.3
cdef float expiration_time = 0.5
cdef int no_of_divisions = 30
cdef float risk_free_rate = 0.05
cdef float strike_price = 40
cdef float u = exp(volatility*sqrt(2*expiration_time/(float(no_of_divisions))))
cdef float d =1/u
cdef float R = exp(risk_free_rate*expiration_time/(float(no_of_divisions)))
cdef float pu = pow((sqrt(R) - sqrt(d))/(sqrt(u)-sqrt(d)), 2)
cdef float pd = pow((sqrt(u)-sqrt(R))/(sqrt(u)-1/sqrt(u)),2)

cache = {}
cdef float american_call(int k, int i, float price) except? -2: 
    ins = (k, i, price)
    if ins in cache:
        return cache[ins]
    if k == no_of_divisions:
        return max(0.0, price-strike_price)
    else:
        val = max(price-strike_price,(pu*american_call(k+1, i+1, price*u)+(1-pu-pd)*american_call(k+1, i, price)+pd*american_call(k+1, i-1, price*d))/R)
        cache[ins] = val
        return val
    
# print(memoize(american_call(0, 0, 50)))
print((american_call(0, 0, 50)))
t2 = time.time()
print("Seconds since epoch =", round(t2-t1, 3))

UsageError: Cell magic `%%cython` not found.


In [12]:
%%cython
import time
from libc.math cimport sqrt, exp, pow

# cdef float american_call(int k, int i, float price) except? -2: 
#     ins = (k, i, price)
#     if ins in cache:
#         return cache[ins]
#     if k == no_of_divisions:
#         return max(0.0, price-strike_price)
#     else:
#         val = max(price-strike_price,(pu*american_call(k+1, i+1, price*u)+(1-pu-pd)*american_call(k+1, i, price)+pd*american_call(k+1, i-1, price*d))/R)
#         cache[ins] = val
#         return val
cache = {}

cdef class Option:
    
    cdef float volatility
    cdef float expiration_time
    cdef int no_of_divisions 
    cdef float risk_free_rate
    cdef float strike_price 
    cdef float u
    cdef float d 
    cdef float R
    cdef float pu
    cdef float pd 
   
    
    def __cinit__(self, float strike, float vol, float exp_time, float r=0.01):
       
        self.volatility = vol
        self.expiration_time = exp_time
        self.no_of_divisions = 25
        self.risk_free_rate = r
        self.strike_price = strike
        self.u = exp(self.volatility*sqrt(2*self.expiration_time/(float(self.no_of_divisions))))
        self.d =1/self.u
        self.R = exp(self.risk_free_rate*self.expiration_time/(float(self.no_of_divisions)))
        self.pu = pow((sqrt(self.R) - sqrt(self.d))/(sqrt(self.u)-sqrt(self.d)), 2)
        self.pd = pow((sqrt(self.u)-sqrt(self.R))/(sqrt(self.u)-1/sqrt(self.u)),2)
    
    def call_(self, int k, int i, float price): 
        global cache
        ins = (k, i, price)
        if ins in cache:
            return cache[ins]
        if k == self.no_of_divisions:
            return max(0.0, price-self.strike_price)
        else:
            val = max(price-self.strike_price,(self.pu*self.call(k+1, i+1, price*self.u)+(1-self.pu-self.pd)*self.call(k+1, i, price)+self.pd*self.call(k+1, i-1, price*self.d))/self.R)
            cache[ins] = val
            return val
        
    def put_(self, int k, int i, float price): 
        global cache
        ins = (k, i, price)
        if ins in cache:
            return cache[ins]
        if k == self.no_of_divisions:
            return max(0.0, -(price-self.strike_price))
        else:
            val = max(-price+self.strike_price,(self.pu*self.put(k+1, i+1, price*self.u)+(1-self.pu-self.pd)*self.put(k+1, i, price)+self.pd*self.put(k+1, i-1, price*self.d))/self.R)
            cache[ins] = val
            return val

def call(float price, float strike, float vol, float exp_time, float r=0.01):
    opt = Option(strike, vol, exp_time, r=r)
    return opt.call_(0, 0, price)
   

opt = Option(41, 0.3, 0.5)
t1 = time.time()
print(opt.call(0, 0, 50))
t2 = time.time()
print("Seconds since epoch =", round(t2-t1, 3))
del(opt)

10.068251816520748
Seconds since epoch = 0.004


In [ ]:
import functools
from math import sqrt, exp
t1 = time.time()
volatility = 0.3
expiration_time = 0.5
no_of_divisions = 30
risk_free_rate = 0.05
strike_price = 40
u = exp(volatility*sqrt(2*expiration_time/(float(no_of_divisions))))
d =1/u
R = exp(risk_free_rate*expiration_time/(float(no_of_divisions)))
pu = pow((sqrt(R) - sqrt(d))/(sqrt(u)-sqrt(d)), 2)
pd = pow((sqrt(u)-sqrt(R))/(sqrt(u)-1/sqrt(u)),2)

@functools.lru_cache()
def american_call_python(k, i , price):
    if k == no_of_divisions:
        return max(0.0, price-strike_price)
    else:
        return max(price-strike_price,(pu*american_call_python(k+1, i+1, price*u)+(1-pu-pd)*american_call_python(k+1, i, price)+pd*american_call_python(k+1, i-1, price*d))/R)

print(american_call_python(0, 0, 50))
t2 = time.time()
print("Seconds since epoch =", t2-t1)

In [ ]:
cache = {(1,2):3} 
if (1,2) in cache:
    print(cache[(1,2)])

In [15]:
cache[(2,3)] = 2
cache

{(1, 2): 3, (2, 3): 2}